In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Loading SIPI Dataset

SIPI dataset was preprocessed using prepare_dataset notebook, so these three folders already contain Detritus/Non-Detritus images.

In [3]:
DatasetName = 'DatasetSIPI'

train_dir = DatasetName+'/train'
validation_dir =  DatasetName+'/val'
test_dir = DatasetName+'/test'

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

The three datasets are loaded using keras preprocessing method *image_dataset_from_directory*. Both the batch size and the image size hyperparameters where tested using different values. 

In [5]:
train_dataset = image_dataset_from_directory(train_dir,
shuffle=True,
batch_size=BATCH_SIZE,
image_size=IMG_SIZE)

validation_dataset = image_dataset_from_directory(validation_dir,
shuffle=True,
batch_size=BATCH_SIZE,
image_size=IMG_SIZE)

test_dataset = image_dataset_from_directory(test_dir,
shuffle=True,
batch_size=BATCH_SIZE,
image_size=IMG_SIZE)

Found 230883 files belonging to 2 classes.


2021-07-13 20:21:20.327158: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-13 20:21:20.425226: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2021-07-13 20:21:20.426248: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:81:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-07-13 20:21:20.426278: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-13 20:21:20.756252: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic libra

Found 49474 files belonging to 2 classes.
Found 49475 files belonging to 2 classes.


# Training

In [6]:
# Hyperparamers to be used in all models
base_learning_rate = 0.0001
IMG_SHAPE = IMG_SIZE + (3,)
TRAINING_EPOCHS = 20

# MobileNet model

A MobileNet model is loaded and modified so it can be used to predict Detritus images..

In [7]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights=None)

In [8]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

2021-07-13 20:22:13.935578: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-13 20:22:13.954232: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2100090000 Hz


(32, 5, 5, 1280)


In [9]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [10]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 1)


In [11]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

inputs = tf.keras.Input(shape=(160, 160, 3))
x = preprocess_input(inputs)
x = base_model(x)
x = global_average_layer(x)
outputs = prediction_layer(x)
mobilenet_model = tf.keras.Model(inputs, outputs)

# DenseNet model

In [12]:
base_model = tf.keras.applications.DenseNet201(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights=None)

In [13]:
preprocess_input = tf.keras.applications.densenet.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

inputs = tf.keras.Input(shape=(160, 160, 3))
x = preprocess_input(inputs)
x = base_model(x)
x = global_average_layer(x)
outputs = prediction_layer(x)
densenet_model = tf.keras.Model(inputs, outputs)

# VGG model

In [14]:
base_model = tf.keras.applications.VGG19(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights=None)

In [15]:
preprocess_input = tf.keras.applications.vgg19.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

inputs = tf.keras.Input(shape=(160, 160, 3))
x = preprocess_input(inputs)
x = base_model(x)
x = global_average_layer(x)
outputs = prediction_layer(x)
vgg_model = tf.keras.Model(inputs, outputs)

# Inception Resnet model

In [16]:
base_model = tf.keras.applications.InceptionResNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights=None)

In [17]:
preprocess_input = tf.keras.applications.inception_resnet_v2.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

inputs = tf.keras.Input(shape=(160, 160, 3))
x = preprocess_input(inputs)
x = base_model(x)
x = global_average_layer(x)
outputs = prediction_layer(x)
inception_resnet_model = tf.keras.Model(inputs, outputs)

# Training models

In [18]:
def train_model(model, epochs):
    print("Training model, epochs: ", epochs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    history = model.fit(train_dataset,
                    epochs=epochs,
                    validation_data=validation_dataset)
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    training_loss = history.history['loss']
    val_loss = history.history['val_loss']

    loss, accuracy = model.evaluate(test_dataset)
    print('Test accuracy :', accuracy)
    
    return acc, val_acc, training_loss, val_loss

In [19]:
def show_plot(acc, val_acc, loss, val_loss):
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,1.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

We test each of the 4 models from scratch. Comparing them by the testing accuracy and analyzing their Training and Validation Accuracy/Loss along the training epochs

In [ ]:
acc, val_acc, loss, val_loss = train_model(mobilenet_model, TRAINING_EPOCHS)
show_plot(acc, val_acc, loss, val_loss)

Training model, epochs:  20
Epoch 1/20


/home/dmansilla/anaconda3/envs/python37/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


7216/7216 [==============================] - 5033s 697ms/step - loss: 0.1653 - accuracy: 0.9324 - val_loss: 0.3136 - val_accuracy: 0.8085
Epoch 2/20
7216/7216 [==============================] - 4812s 667ms/step - loss: 0.1033 - accuracy: 0.9594 - val_loss: 0.1110 - val_accuracy: 0.9595
Epoch 3/20
7216/7216 [==============================] - 4785s 663ms/step - loss: 0.0906 - accuracy: 0.9645 - val_loss: 0.1095 - val_accuracy: 0.9521
Epoch 4/20
7216/7216 [==============================] - 4748s 658ms/step - loss: 0.0823 - accuracy: 0.9677 - val_loss: 0.1180 - val_accuracy: 0.9519
Epoch 5/20
7216/7216 [==============================] - 4722s 654ms/step - loss: 0.0759 - accuracy: 0.9702 - val_loss: 0.1109 - val_accuracy: 0.9495
Epoch 6/20
7216/7216 [==============================] - 4705s 652ms/step - loss: 0.0695 - accuracy: 0.9728 - val_loss: 0.1045 - val_accuracy: 0.9583
Epoch 7/20
7216/7216 [==============================] - 4691s 650ms/step - loss: 0.0640 - accuracy: 0.9747 - val_loss

In [ ]:
acc, val_acc, loss, val_loss = train_model(densenet_model, TRAINING_EPOCHS)
show_plot(acc, val_acc, loss, val_loss)

In [ ]:
acc, val_acc, loss, val_loss = train_model(inception_resnet_model, TRAINING_EPOCHS)
show_plot(acc, val_acc, loss, val_loss)

| Model | Training Accuracy | Testing Accuracy |
| --- | --- | --- |
| MobileNet | .9833 | .8997 |
| DenseNet | .9921 | .9438 |
| Inception Resnet | .9949 | .9383 |

After comparing them we choose MobileNet as it demands a smaller number of parameters to train. We use techniques in order to improve Training and Validation accuracy.

# Transfer learning with freezing

We load the model again but with pre-set imagenet weights.

In [ ]:
TRAINING_EPOCHS = 40

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

preprocess_input = tf.keras.applications.densenet.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

inputs = tf.keras.Input(shape=(160, 160, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
acc, val_acc, loss, val_loss = train_model(model, TRAINING_EPOCHS)
show_plot(acc, val_acc, loss, val_loss)

We achieve around 92% training accuracy and around 91% testing accuracy.

# Transfer learning without freezing

We load the model again but with pre-set imagenet weights.

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

preprocess_input = tf.keras.applications.densenet.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

inputs = tf.keras.Input(shape=(160, 160, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
acc, val_acc, loss, val_loss = train_model(model, TRAINING_EPOCHS)
show_plot(acc, val_acc, loss, val_loss)

We achieve around 99.5% training accuracy and around 95% testing accuracy.